> 需要安装 bt 包

```bash
! pip install backtrader --upgrade --user -i https://pypi.tuna.tsinghua.edu.cn/simple
```

In [31]:
#!pip install backtrader[plotting]  --upgrade --user -i https://pypi.tuna.tsinghua.edu.cn/simple

> FileNotFoundError: [Errno 2] No such file or directory: 'AAPL'

In [12]:
#!pip install yfinance  --upgrade --user -i https://pypi.tuna.tsinghua.edu.cn/simple

In [9]:
import datetime

import numpy as np
import backtrader as bt

# 子策略：行业轮动策略

In [4]:
class SectorRotation(bt.Strategy):
    params = (('momentum_period', 126),)  # 6个月动量

    def __init__(self):
        # 计算每个行业的动量
        self.ranks = {d: bt.indicators.Momentum(d.close, period=self.p.momentum_period) 
                      for d in self.datas}

    def next(self):
        # 仅由父策略调用，不自主交易
        pass

    def get_sector_weights(self):
        """返回行业权重字典"""
        # 获取动量排名
        momentum_values = [self.ranks[d][0] for d in self.datas]
        ranked_data = sorted(zip(self.datas, momentum_values), 
                            key=lambda x: x[1], reverse=True)

        # 等权重配置前3行业
        top_sectors = [d for d, _ in ranked_data[:3]]
        return {d: 1.0/len(top_sectors) for d in top_sectors}

# 父策略：风险平价策略

In [5]:
class RiskParity(bt.Strategy):
    params = (
        ('rebalance_days', 21),  # 每月调仓
        ('target_risk', 0.1),    # 目标波动率
    )

    def __init__(self):
        self.counter = 0
        # 创建子策略实例
        self.child_strategy = SectorRotation()

        # 风险平价需要的数据：大类资产
        self.assets = {
            'stocks': self.data0,
            'bonds': self.data1,
            'commodities': self.data2
        }
        self.asset_vol = {}  # 存储波动率

    def next(self):
        self.counter += 1

        # 每月执行一次调仓
        if self.counter % self.p.rebalance_days == 0:
            # 1. 计算大类资产波动率 (简化版)
            for name, data in self.assets.items():
                returns = np.diff(np.log(data.close.get(size=252)))
                self.asset_vol[name] = np.std(returns) * np.sqrt(252)

            # 2. 计算风险平价权重
            inv_vol = {name: 1/vol for name, vol in self.asset_vol.items()}
            total_inv_vol = sum(inv_vol.values())
            risk_weights = {name: iv/total_inv_vol for name, iv in inv_vol.items()}

            # 3. 获取子策略行业权重
            sector_weights = self.child_strategy.get_sector_weights()

            # 4. 计算最终权重
            stock_weight = risk_weights['stocks']
            for sector, weight in sector_weights.items():
                target_percent = stock_weight * weight
                current_percent = self.broker.getvalue([sector]) / self.broker.getvalue()
                self.order_target_percent(sector, target_percent)

            # 5. 配置其他大类资产
            self.order_target_percent(self.assets['bonds'], risk_weights['bonds'])
            self.order_target_percent(self.assets['commodities'], risk_weights['commodities'])

# 执行回测

In [ ]:
cerebro = bt.Cerebro()

# 数据范围
start = datetime.datetime(2024,1,1)
end = datetime.datetime(2025,1,1)

# 添加数据 (示例)
# 大类资产
cerebro.adddata(bt.feeds.YahooFinanceData(dataname='SPY', fromdate=start, todate=end))  # 股票
#cerebro.adddata(bt.feeds.YahooFinanceData(dataname='TLT', fromdate=start, todate=end))  # 债券
#cerebro.adddata(bt.feeds.YahooFinanceData(dataname='GSG', fromdate=start, todate=end))  # 商品

# 行业ETF数据
#for ticker in ['XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLRE', 'XLU', 'XLV', 'XLY']:
#    data = bt.feeds.YahooFinanceData(dataname=ticker, fromdate=start, todate=end)
#    cerebro.adddata(data)

cerebro.addstrategy(RiskParity)
cerebro.broker.set_cash(100000)
results = cerebro.run()

FileNotFoundError: [Errno 2] No such file or directory: 'SPY'

In [19]:
from datetime import datetime
import backtrader as bt

import yfinance as yf
print(yf.__version__)

class SmaCross(bt.SignalStrategy):
    def __init__(self):
        sma1, sma2 = bt.ind.SMA(period=10), bt.ind.SMA(period=30)
        crossover = bt.ind.CrossOver(sma1, sma2)
        self.signal_add(bt.SIGNAL_LONG, crossover)

cerebro = bt.Cerebro()
cerebro.addstrategy(SmaCross)

#data0 = bt.feeds.YahooFinanceData(dataname='MSFT', fromdate=datetime(2011, 1, 1),
#                                  todate=datetime(2012, 12, 31))
SPY=yf.download('SPY', '2025-07-06', '2025-07-24', auto_adjust=True)

SPY
#data0 = bt.feeds.PandasData(dataname=SPY)
#cerebro.adddata(data0)

#cerebro.run()
#cerebro.plot()

0.2.65


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['SPY']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,SPY,SPY,SPY,SPY,SPY,SPY
Date,,,,,,
